In [1]:
%useLatestDescriptors
%use lets-plot
%use ggdsl(0.1.4-dev-31)

@file:DependsOn("org.apache.commons:commons-math3:3.6.1")

In [2]:
import org.apache.commons.math3.distribution.MultivariateNormalDistribution

In [3]:
val cov0 : Array<DoubleArray> = arrayOf(doubleArrayOf(1.0, -.8),
                                        doubleArrayOf(-.8, 1.0))

val cov1 : Array<DoubleArray> = arrayOf(doubleArrayOf(1.0, .8),
                                        doubleArrayOf(.8, 1.0))

val cov2 : Array<DoubleArray> = arrayOf(doubleArrayOf(10.0, .1),
                                        doubleArrayOf(.1, .1))


In [4]:
val n = 400

val means0 : DoubleArray = doubleArrayOf(-2.0, 0.0)
val means1 : DoubleArray = doubleArrayOf(2.0, 0.0)
val means2 : DoubleArray = doubleArrayOf(0.0, 1.0)

val xy0 = MultivariateNormalDistribution(means0, cov0).sample(n)
val xy1 = MultivariateNormalDistribution(means1, cov1).sample(n)
val xy2 = MultivariateNormalDistribution(means2, cov2).sample(n)


In [5]:
val data = mapOf(
    "x" to (xy0.map { it[0] } + xy1.map { it[0] } + xy2.map { it[0] }).toList(),
    "y" to (xy0.map { it[1] } + xy1.map { it[1] } + xy2.map { it[1] }).toList()
)


In [6]:
val p = letsPlot(data) {x="x"; y="y"} + ggsize(600,300) +
        geomPoint(color="black", alpha=.1)
p

In [7]:
val xSrc = source<Double>("x")
val ySrc = source<Double>("y")

In [8]:
plot(data) {
    points { 
        x(xSrc)
        y(ySrc)
        color(Color.BLACK)
        alpha(.1)
    }
    layout.size = 600 to 300
}

In [9]:
// Basic density 
p + geomDensity2D(color="red")

In [10]:
plot(data) {
    points { 
        x(xSrc)
        y(ySrc)
        color(Color.BLACK)
        alpha(.1)
    }
    density2D(xSrc, ySrc) {
        borderLineColor(Color.RED)
    }
    layout.size = 600 to 300
}

In [11]:
// Set contour color by level
//  - change defailt position and size of colorbar
p + geomDensity2D {color="..level.."} +
    scaleColorGradient(low="dark_green", high="yellow", guide=guideColorbar(barHeight=10, barWidth=300)) +
    theme().legendPositionBottom()

In [12]:
plot(data) {
    points { 
        x(xSrc)
        y(ySrc)
        color(Color.BLACK)
        alpha(.1)
    }
    density2D(xSrc, ySrc) {
        borderLineColor(Stat.LEVEL.scaled(continuous( 
            rangeLimits = Color.fromName("dark_green") to Color.YELLOW
        ))).with { 
            legend {
                type = colorBar(barHeight = 10.0, barWidth = 300.0 )
            }
        }
    }
    layout{
        size = 600 to 300
        theme{
            legend.position = LegendPosition.Bottom
        }
    }
}

### Filling contours by level

In [13]:
val p1 = letsPlot(data) {x="x"; y="y"} + ggsize(600,300)

// Filled polygons are not always working well - note missing polygons in the middle. 
p1 + geomPolygon(stat=org.jetbrains.letsPlot.Stat.density2D()) {fill="..level.."} + coordFixed()

In [14]:
// 'geomDensity2DFilled' is not dependent on poligons order and works a lot better
p1 + geomDensity2DFilled {fill="..level.."}

In [15]:
plot(data) {
    density2DFilled(xSrc, ySrc) {
        fillColor(Stat.LEVEL)
    }
    layout{
        size = 600 to 300
    }
}

### geomBin2D is another way to plot density

In [16]:
p1 + geomBin2D()

In [17]:
plot(data) {
    bin2D(xSrc, ySrc) { }
    layout{
        size = 600 to 300
    }
}

In [18]:
// Adjust the tile size - make them square and bigger.
// Show density instead of count.
p1 + geomBin2D(binWidth=1 to 1) {fill="..density.."}

In [19]:
plot(data) {
    bin2D(xSrc, ySrc, bins = Bins2D.byWidth(1.0, 1.0)) { 
        fillColor(Stat.DENSITY)
    }
    layout{
        size = 600 to 300
    }
}